Default configuration

In [4]:
# Define paths to model files
import os
MODELS_DIR = 'models\\'
if not os.path.exists(MODELS_DIR):
    os.mkdir(MODELS_DIR)
MODEL_TF = MODELS_DIR + 'model'
MODEL_NO_QUANT_TFLITE = MODELS_DIR + 'model_no_quant.tflite'
MODEL_TFLITE = MODELS_DIR + 'model.tflite'
MODEL_TFLITE_MICRO = MODELS_DIR + 'model.cc'

In [3]:

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
# from cv2 import imwrite,imread

print(tf.__version__)

ModuleNotFoundError: No module named 'tensorflow'

<p>Normalize data</p>


<p> importante notar que se os dados devem ser pré-processados, para então ser preditos pela NN<p/>

In [2]:
fashion_mnist = tf.keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()
# Normalize the data
train_images = train_images.astype(np.float32) / 255
test_images = test_images.astype(np.float32) / 255
# de 0 a 1 na escala 255 ou 8 bits
train_images = np.expand_dims(train_images, -1)
test_images = np.expand_dims(test_images, -1)

<p>Modele a rede neural</p>

In [7]:
from tensorflow.keras import layers
input_shape = (28, 28, 1)
num_classes = 10
model = tf.keras.Sequential(
    [
        tf.keras.Input(shape=input_shape),
        layers.Conv2D(6, kernel_size=(3, 3), activation="relu"),
        layers.Conv2D(6, kernel_size=(3, 3), activation="relu"),
        layers.Flatten(),
        layers.Dense(num_classes, activation="softmax"), # RESULTADO SÃO 10 TIPOS DE ROUPA
    ]
)
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 6)         60        
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 6)         330       
                                                                 
 flatten (Flatten)           (None, 3456)              0         
                                                                 
 dense (Dense)               (None, 10)                34570     
                                                                 
Total params: 34,960
Trainable params: 34,960
Non-trainable params: 0
_________________________________________________________________


<p>treine a rede</p>

In [8]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
model.fit(train_images, train_labels, epochs=20, validation_split=0.2)

Epoch 1/20


c:\Users\Nikolaos\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\backend.py:5582: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


1500/1500 [==============================] - 13s 8ms/step - loss: 0.4791 - accuracy: 0.8301 - val_loss: 0.3903 - val_accuracy: 0.8558
Epoch 2/20
1500/1500 [==============================] - 12s 8ms/step - loss: 0.3509 - accuracy: 0.8746 - val_loss: 0.3491 - val_accuracy: 0.8763
Epoch 3/20
1500/1500 [==============================] - 12s 8ms/step - loss: 0.3181 - accuracy: 0.8851 - val_loss: 0.3360 - val_accuracy: 0.8784
Epoch 4/20
1500/1500 [==============================] - 12s 8ms/step - loss: 0.2885 - accuracy: 0.8951 - val_loss: 0.3101 - val_accuracy: 0.8897
Epoch 5/20
1500/1500 [==============================] - 12s 8ms/step - loss: 0.2674 - accuracy: 0.9020 - val_loss: 0.3170 - val_accuracy: 0.8886
Epoch 6/20
1500/1500 [==============================] - 12s 8ms/step - loss: 0.2502 - accuracy: 0.9082 - val_loss: 0.3056 - val_accuracy: 0.8956
Epoch 7/20
1500/1500 [==============================] - 12s 8ms/step - loss: 0.2351 - accuracy: 0.9149 - val_loss: 0.3040 - val_accuracy: 0.8

<p>salve o modelo<p/>

In [9]:
# Save the model to disk
model.save(MODEL_TF)

INFO:tensorflow:Assets written to: models\model\assets


INFO:tensorflow:Assets written to: models\model\assets


<p> teste a acurácia<p/>

In [10]:
test_loss, test_acc = model.evaluate(test_images,  test_labels)

print('\nTest accuracy:', test_acc)

 34/313 [==>...........................] - ETA: 0s - loss: 0.3446 - accuracy: 0.8952

c:\Users\Nikolaos\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\backend.py:5582: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


313/313 [==============================] - 1s 3ms/step - loss: 0.3938 - accuracy: 0.8886

Test accuracy: 0.8885999917984009


predição de valores


In [11]:
predictions = model.predict(test_images)
predictions[0]

313/313 [==============================] - 1s 3ms/step


array([1.7340051e-11, 4.3830321e-19, 1.8910325e-11, 5.3509796e-12,
       3.2970025e-15, 8.4616302e-05, 1.2833290e-12, 1.6768759e-05,
       1.6661293e-08, 9.9989855e-01], dtype=float32)

In [10]:
np.argmax(predictions[0])

9

Quantizar o modelo tf_lite

In [14]:
# Convert the model to the TensorFlow Lite format without quantization
converter = tf.lite.TFLiteConverter.from_saved_model(MODEL_TF)
model_no_quant_tflite = converter.convert()

# Save the model to disk
size = open(MODEL_NO_QUANT_TFLITE, "wb").write(model_no_quant_tflite)
print("No quantized model is %d bytes" % size)

No quantized model is 142428 bytes


In [13]:

# Convert the model to the TensorFlow Lite format with quantization
def representative_dataset():
 for input_value in tf.data.Dataset.from_tensor_slices(train_images).batch(1).take(100):
    yield [input_value]
# Set the optimization flag.
converter.optimizations = [tf.lite.Optimize.DEFAULT]

# Provide a representative dataset to ensure we quantize correctly.
converter.representative_dataset = representative_dataset
tflite_model = converter.convert()

tflite_model_size = open(MODEL_TFLITE, "wb").write(tflite_model)
print("Quantized model is %d bytes" % tflite_model_size)

Quantized model is 38616 bytes


<p>Exporte o modelo</p>

In [17]:
# Install xxd if it is not available

# Convert to a C source file, i.e, a TensorFlow Lite for Microcontrollers model
!xxd -i {MODEL_TFLITE} > {MODEL_TFLITE_MICRO}
# Update variable names
REPLACE_TEXT = MODEL_TFLITE.replace('/', '_').replace('.', '_')
!sed -i 's/'{REPLACE_TEXT}'/g_model/g' {MODEL_TFLITE_MICRO}

'sed' n�o � reconhecido como um comando interno
ou externo, um programa oper�vel ou um arquivo em lotes.


teste --ABAIXO--

In [6]:
train_labels.shape

(60000,)

In [36]:
print(train_images.shape)
print(test_images.shape)

(60000, 28, 28, 1)
(10000, 28, 28, 1)
